In [63]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import pickle

## **Load essentials**

In [8]:
model = XGBClassifier()
model.load_model('model.bin')

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [5]:
data = pd.read_csv('creditcard.csv')
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [59]:
y_test = data['Class'][:10000]  # Real values
X_test = data[:10000].drop('Class', axis=1)  # Get 100 data and remove the class column
scaled = scaler.transform(X_test[['Time', 'Amount']])
scaled_df = pd.DataFrame(scaled, columns=['Time', 'Amount'])

# Add the scaled data to the original dataset
X_test = X_test.drop(['Time', 'Amount'], axis=1)
X_test = pd.concat([X_test, scaled_df], axis=1)
X_test.head(2)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Time,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,-1.996583,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-1.996583,-0.342475


## **Test Model**

In [60]:
y_preds = model.predict(X_test)

# Create DataFrame with actual and predicted datas
results = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_preds
})

results.head()

,Actual,Predicted
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [61]:
# Actual Fraud data with predicted data
results[results['Actual'] == 1]

,Actual,Predicted
541,1,1
623,1,1
4920,1,1
6108,1,1
6329,1,1
6331,1,1
6334,1,1
6336,1,1
6338,1,1
6427,1,1


✅ All Actual Fraud transactions predicted succesfully.

In [62]:
# Predicted Fraud data with actual data
results[results['Predicted'] == 1]

,Actual,Predicted
460,0,1
472,0,1
541,1,1
623,1,1
730,0,1
4920,1,1
6063,0,1
6108,1,1
6329,1,1
6331,1,1


❌ Some normal transactions predicted as Fraud

In [64]:
score = accuracy_score(y_test, y_preds)
print('Accuracy score: {}'.format(score))

Accuracy score: 0.9987
